# Modelos conceituais

### UniProt = Human Diseases

![](000-mc-er-uniprothd.png)

### Breast Cancer Winconsin (Diagnosis) Data Set

![](001-mc-er-wisconsin.png)

### TB Database

![](003-mc-er-tbdb.png)

### Virus-Host DB

![](004-mc-er-virushost.png)

# Modelos lógicos

Nota: apenas das bases que deram origem a consultas relacionais

### UniProt = Human Diseases

> **Diseases**(<u>Id</u>, Name, Mnemonic, Description)
>
> **DiseasesAnalysis**(Category, Amount)
>
> **DiseasesAnalysisAux**(Category, Amount)
 

### Breast Cancer Winconsin (Diagnosis) Data Set

> **Paciente**(<u>Id</u>, Diagnostico, Raio, Textura, Perimetro, Area, Suavidade, Compacidade, Concavidade, PontosConcavos, Simetria, DimensaoFractal)
>
> **Erro**(<u>IdPaciente</u>, Raio_erro, Textura_erro, Perimetro_erro, Area_erro, Suavidade_erro, Compacidade_erro, Concavidade_erro, PontosConcavos_erro, Simetria_erro, DimensaoFractal_erro)<br>
> -> CHE IdPaciente -> Paciente(Id)


# Código de conversão

### TB Database

Conversão do formato Gene Transfer Format (GTF) para XML:

```cpp
#include <iostream>
#include <fstream>
#include <string>
#include <stdio.h>
#include<string.h>

using namespace std;

//componente para o vertex
struct GFT_LINE{
  string seqname;
  string type;
  string source;
  string feature;
  string start_value;
  string end_value;
  string score ;
  string strand;
  string frame;
  string gene_id;
  string transcript_id;
};

int main()
{
  string line;
  ifstream myfile ("rhodobacter_sphaeroides_2.4.1_1_transcripts.txt");
  ofstream outfile;
  outfile.open ("file.xml");
  outfile<<"<?xml version=\"1.0\"?>"<<endl;
  if (myfile.is_open()){
    GFT_LINE *beforeLine = new GFT_LINE;
    beforeLine->seqname = "";
    beforeLine->type = "";
    GFT_LINE *l;
    while ( getline (myfile,line) ){
      l = new GFT_LINE;
      string word;
      int start = 0;
      int part = 0;
      for(int i = 0; i<line.length(); i++){
        word = line.substr(start,i-start);
        if(word == "Chromosome_"){
          word = line.substr(start,i+1);
          l->seqname = word;
          l->type = line[i+2];
          i +=4;
          start = i;
        }else if(word == "gene_id "){
          start = i+1;
          for(int j= i+2; j < line.length(); j++){
            word = line.substr(start,j-start);
            if(line[j]=='\"'){
              i= j+2;
              start = j+2;
              break;
            }
          }
          i ++;
          start = i;
          l->gene_id = word;
        }else if(word == "transcript_id "){
          start = i+1;
          for(int j= i+2; j < line.length(); j++){
            word = line.substr(start,j-start);
            if(line[j]=='\"'){
              break;
            }
          }
          l->transcript_id = word;
          break;
        }else if(line[i] == '\t'){
           word = line.substr(start,i-start);
           start = i+1;
            if(part == 0){
              l->source = word;
            }else if(part == 1){
              l->feature = word;
            }else if(part == 2){
              l->start_value = word;
            }else if (part == 3){
              l->end_value = word;
            }else if (part == 4){
              l->score = word;
            }else if (part == 5){
              l->strand = word;
            }else if (part == 6){
              l->frame = word;
            }
            part++;
        }
      }
      int novo = 0;
      if(beforeLine->seqname != l->seqname){
        if(beforeLine->seqname != ""){
          outfile << "\t</type >" <<endl;
          outfile << "</" << beforeLine->seqname << ">" << endl;
        }
        outfile << "<" << l->seqname << ">" << endl;
      }
      if(beforeLine->type != l->type || beforeLine->seqname != l->seqname){
        if(beforeLine->seqname != ""){
          outfile << "\t</type >" <<endl;
        }
        outfile << "\t<type value=\""<< l->type << "\">" <<endl;
        novo = 1;
      }else
        novo = 0;
      outfile<<"\t\t<property " ;
      outfile<<"source=\""<< l->source<< "\" ";
      outfile<<"feature=\""<< l->feature<< "\" ";
      outfile<<"start=\""<< l->start_value<< "\" ";
      outfile<<"end=\""<< l->end_value<< "\" ";
      outfile<<"score=\""<< l->score<< "\" ";
      outfile<<"strand=\""<< l->strand<< "\" ";
      outfile<<"frame=\""<< l->frame<< "\" ";
      outfile<<"gene_id=\""<< l->gene_id<< "\" ";
      outfile<<"transcript_id=\""<< l->transcript_id<< "\" />" << endl;
      if((beforeLine->type != l->type && beforeLine->seqname == l->seqname) && !novo)
        outfile << "\t</type >" <<endl;
      beforeLine = l;
    }
    outfile << "\t</type >" <<endl;
    outfile << "</" << l->seqname << ">" << endl;
    myfile.close();
  }

  else cout << "Unable to open file";

    return 0;
}
```

# SQL

Os scripts de criação de tabela e normalização dos dados estão disponíveis nos notebooks em anexo. Seguem abaixo as principais consultas realizadas nas bases de dados.

### UniProt - Human Diseases

#### Doenças mais comuns

Esta query em SQL, retorna as informações das 5 categorias de doenças mais comuns relacionadas a algum tipo de proteina

```sql
SELECT * FROM DiseasesAnalysis
ORDER BY amount DESC
LIMIT 11;
```

#### Suporte

Esta query em SQL, retorna o valor do suporte de cada categoria de doença, ou seja, a probabilidade de dada uma doença qualquer, ela ser dessa categoria específica

```sql
SELECT Category, SUM(CAST(Amount AS DOUBLE) / (SELECT COUNT(*) numberofdiseases
                                               FROM Diseases)) suporte
FROM DiseasesAnalysis
GROUP BY category
ORDER BY suporte DESC;
```

#### Duas categorias na mesma doença

Uma informação interessante a se pensar com relação aos dados de doençar é a quantidade de doenças que pertencem a duas categorias simultaneamente. Essa query mostra essas ocorrências pra algumas das categorias mais comuns escolhidas para analise

```sql
SELECT 'syndrome', 'deficiency', COUNT(*) occurrences
FROM Diseases D
WHERE D.name LIKE '%syndrome%' AND
      D.name LIKE '%deficiency%';
      
SELECT 'syndrome', 'mental', COUNT(*) occurrences
FROM Diseases D
WHERE D.name LIKE '%syndrome%' AND
      D.name LIKE '%mental%';
      
SELECT 'syndrome', 'congenital', COUNT(*) occurrences
FROM Diseases D
WHERE D.name LIKE '%syndrome%' AND
      D.name LIKE '%congenital%';
      
SELECT 'mental', 'deficiency', COUNT(*) occurrences
FROM Diseases D
WHERE D.name LIKE '%mental%' AND
      D.name LIKE '%deficiency%';
      
SELECT 'autosomal dominant', 'deafness', COUNT(*) occurrences
FROM Diseases D
WHERE D.name LIKE '%autosomal dominant%' AND
      D.name LIKE '%deafness%';
      
SELECT 'autosomal dominant', 'syndrome', COUNT(*) occurrences
FROM Diseases D
WHERE D.name LIKE '%autosomal dominant%' AND
      D.name LIKE '%syndrome%';
      
SELECT 'autosomal dominant', 'mental', COUNT(*) occurrences
FROM Diseases D
WHERE D.name LIKE '%autosomal dominant%' AND
      D.name LIKE '%mental%';
      
SELECT 'X-linked', 'dysplasia', COUNT(*) occurrences
FROM Diseases D
WHERE D.name LIKE '%X-linked%' AND
      D.name LIKE '%dysplasia%';
      
SELECT 'hereditary', 'syndrome', COUNT(*) occurrences
FROM Diseases D
WHERE D.name LIKE '%hereditary%' AND
      D.name LIKE '%syndrome%';
```

#### Confiança

Outra informação possivelmente mais importante que as ocorrências de duas categorias ao mesmo tempo, é a confiança dessas associações: a probabilidade de que ambas ocorram ao mesmo tempo

```sql
SELECT 'syndrome', 'deficiency', CAST((SELECT COUNT(*) occurrences
                                       FROM Diseases D
                                       WHERE D.name LIKE '%syndrome%' AND
                                             D.name LIKE '%deficiency%') AS DOUBLE) / (SELECT amount
                                                                                       FROM DiseasesAnalysis
                                                                                       WHERE category = 'syndrome') confiança;
                                                             
SELECT 'syndrome', 'mental', CAST((SELECT COUNT(*) occurrences
                                   FROM Diseases D
                                   WHERE D.name LIKE '%syndrome%' AND
                                         D.name LIKE '%mental%') AS DOUBLE) / (SELECT amount
                                                                               FROM DiseasesAnalysis
                                                                               WHERE category = 'syndrome') confiança;
                                                             
SELECT 'syndrome', 'congenital', CAST((SELECT COUNT(*) occurrences
                                       FROM Diseases D
                                       WHERE D.name LIKE '%syndrome%' AND
                                             D.name LIKE '%congenital%') AS DOUBLE) / (SELECT amount
                                                                                       FROM DiseasesAnalysis
                                                                                       WHERE category = 'syndrome') confiança;
                                                             
SELECT 'mental', 'deficiency', CAST((SELECT COUNT(*) occurrences
                                     FROM Diseases D
                                     WHERE D.name LIKE '%mental%' AND
                                           D.name LIKE '%deficiency%') AS DOUBLE) / (SELECT amount
                                                                                     FROM DiseasesAnalysis
                                                                                     WHERE category = 'mental') confiança;
                                                             
SELECT 'autosomal dominant', 'deafness', CAST((SELECT COUNT(*) occurrences
                                               FROM Diseases D
                                               WHERE D.name LIKE '%autosomal dominant%' AND
                                                     D.name LIKE '%deafness%') AS DOUBLE) / (SELECT amount
                                                                                             FROM DiseasesAnalysis
                                                                                             WHERE category = 'autosomal dominant') confiança;
                                                             
SELECT 'autosomal dominant', 'syndrome', CAST((SELECT COUNT(*) occurrences
                                               FROM Diseases D
                                               WHERE D.name LIKE '%autosomal dominant%' AND
                                                     D.name LIKE '%syndrome%') AS DOUBLE) / (SELECT amount
                                                                                             FROM DiseasesAnalysis
                                                                                             WHERE category = 'autosomal dominant') confiança;
                                                             
SELECT 'autosomal dominant', 'mental', CAST((SELECT COUNT(*) occurrences
                                             FROM Diseases D
                                             WHERE D.name LIKE '%autosomal dominant%' AND
                                                   D.name LIKE '%mental%') AS DOUBLE) / (SELECT amount
                                                                                         FROM DiseasesAnalysis
                                                                                         WHERE category = 'autosomal dominant') confiança;
                                                             
SELECT 'X-linked', 'dysplasia', CAST((SELECT COUNT(*) occurrences
                                      FROM Diseases D
                                      WHERE D.name LIKE '%X-linked%' AND
                                            D.name LIKE '%dysplasia%') AS DOUBLE) / (SELECT amount
                                                                                     FROM DiseasesAnalysis
                                                                                     WHERE category = 'X-linked') confiança;
```

#### Probabilidade

Outra informação que podemos retirar comparar as probabilidade de uma doença ser de uma categoria mais especifica em relação a uma menos especifica.

```sql
SELECT CAST(SELECT SUM(amount) from DiseasesAnalysis 
            WHERE category LIKE '%autosomal dominant%' AS DOUBLE)/
            CAST((SELECT count(*) NAME 
                  FROM Diseases 
                  WHERE name LIKE '%autosomal%') AS DOUBLE) autosomalDominant_autosomal,
        CAST(SELECT SUM(amount) from DiseasesAnalysis 
            WHERE category LIKE '%autosomal dominant%' AS DOUBLE)/
            CAST((SELECT count(*) NAME 
                  FROM Diseases) AS DOUBLE) autosomalDominant_total;

SELECT (CAST(SELECT SUM(amount) from DiseasesAnalysis 
            WHERE category LIKE '%autosomal recessive%' AS DOUBLE)/
            CAST((SELECT count(*) NAME 
                  FROM Diseases 
                  WHERE name LIKE '%autosomal%') AS DOUBLE))autosomalRecessive_autosomal,
       (CAST(SELECT SUM(amount) from DiseasesAnalysis 
            WHERE category LIKE '%autosomal recessive%' AS DOUBLE)/
            CAST((SELECT count(*) NAME 
                  FROM Diseases) AS DOUBLE)) autosomalRecessive_total;

SELECT (CAST(SELECT SUM(amount) from DiseasesAnalysis 
            WHERE category LIKE '%x-linked%' AS DOUBLE)/
            CAST((SELECT count(*) NAME 
                  FROM Diseases 
                  WHERE name LIKE '%linked%') AS DOUBLE)) X_LINKED,
       (CAST(SELECT SUM(amount) from DiseasesAnalysis 
            WHERE category LIKE '%x-linked%' AS DOUBLE)/
            CAST((SELECT count(*) NAME 
                  FROM Diseases) AS DOUBLE)) X_TOTAL;
                  
SELECT (CAST(SELECT SUM(amount) from DiseasesAnalysis 
            WHERE category LIKE '%y-linked%' AS DOUBLE)/
            CAST((SELECT count(*) NAME 
                  FROM Diseases 
                  WHERE name LIKE '%linked%') AS DOUBLE)) Y_LINKED,
       (CAST(SELECT SUM(amount) from DiseasesAnalysis 
            WHERE category LIKE '%y-linked%' AS DOUBLE)/
            CAST((SELECT count(*) NAME 
                  FROM Diseases) AS DOUBLE)) Y_TOTAL;

```


### Breast Cancer Winconsin (Diagnosis) Data Set

Consulta simples para descobrirmos as medias de valores das amostras diagnosticadas como benigmas ou malignas, para termos ideia de alguns limiares usados para analise dos dados

```sql
SELECT AVG(Raio) AS raio_media, AVG(Textura) AS textura_media, AVG(Perimetro) AS perimetro_media, AVG(Area) AS area_media,
       AVG(Suavidade) AS suavidade_media, AVG(Compacidade) AS compacidade_media, AVG(Concavidade) AS concavidade_media,
       AVG(PontosConcavos) AS pontosconcavos_media, AVG(Simetria) AS simetria_media, 
       AVG(DimensaoFractal) AS dimensaofractal_media
FROM Paciente
WHERE Diagnostico = 'M';

SELECT AVG(Raio) AS raio_media, AVG(Textura) AS textura_media, AVG(Perimetro) AS perimetro_media, AVG(Area) AS area_media,
       AVG(Suavidade) AS suavidade_media, AVG(Compacidade) AS compacidade_media, AVG(Concavidade) AS concavidade_media,
       AVG(PontosConcavos) AS pontosconcavos_media, AVG(Simetria) AS simetria_media, 
       AVG(DimensaoFractal) AS dimensaofractal_media
FROM Paciente
WHERE Diagnostico = 'B';
```

Outra consulta simples em que recuperamos os valores maximos e minimos para compararmos com as medias obtidas acima, onde talvez possamos obter alguns dados úteis

```sql
SELECT MAX(Raio) AS raio_max, MAX(Textura) AS textura_max, MAX(Perimetro) AS perimetro_max, MAX(Area) AS area_max,
       MAX(Suavidade) AS suavidade_max, MAX(Compacidade) AS compacidade_max, MAX(Concavidade) AS concavidade_max,
       MAX(PontosConcavos) AS pontosconcavos_max, MAX(Simetria) AS simetria_max, 
       MAX(DimensaoFractal) AS dimensaofractal_max
FROM Paciente
WHERE Diagnostico = 'M';

SELECT MAX(Raio) AS raio_max, MAX(Textura) AS textura_max, MAX(Perimetro) AS perimetro_max, MAX(Area) AS area_max,
       MAX(Suavidade) AS suavidade_max, MAX(Compacidade) AS compacidade_max, MAX(Concavidade) AS concavidade_max,
       MAX(PontosConcavos) AS pontosconcavos_max, MAX(Simetria) AS simetria_max, 
       MAX(DimensaoFractal) AS dimensaofractal_max
FROM Paciente
WHERE Diagnostico = 'B';
```

Consultas que checas quantas amostras benigmas possuem determinada característica maior que a média de amostras malignas, para determinar se tal característica é um fator importante para indicar se é benigno ou maligno.

```sql
SELECT COUNT(*)
FROM Paciente Pa
WHERE Pa.Diagnostico = 'B' AND Pa.Area > (SELECT AVG(P.Area) FROM Paciente P WHERE P.Diagnostico = 'M');

SELECT COUNT(*)
FROM Paciente Pa
WHERE Pa.Diagnostico = 'B' AND Pa.Perimetro > (SELECT AVG(P.Perimetro) FROM Paciente P WHERE P.Diagnostico = 'M');

SELECT COUNT(*)
FROM Paciente Pa
WHERE Pa.Diagnostico = 'B' AND Pa.Concavidade > (SELECT AVG(P.Concavidade) FROM Paciente P WHERE P.Diagnostico = 'M');

SELECT COUNT(*)
FROM Paciente Pa
WHERE Pa.Diagnostico = 'B' AND Pa.Compacidade > (SELECT AVG(P.Compacidade) FROM Paciente P WHERE P.Diagnostico = 'M');

SELECT COUNT(*)
FROM Paciente Pa
WHERE Pa.Diagnostico = 'B' AND Pa.PontosConcavos > (SELECT AVG(P.PontosConcavos) FROM Paciente P WHERE P.Diagnostico = 'M');
```

Consulta para verificar uma amostra considerando os valores obtidos nos 5 fatores acima, de forma a identificar quantas amostras benignas fogem dos padrões identificados

```sql
SELECT COUNT(*)
FROM Paciente
WHERE Diagnostico = 'B' AND Area > 978.37 AND Perimetro > 115.36 AND Concavidade > 0.16 AND Compacidade >  0.145 AND 
      PontosConcavos > 0.088
```

Cálculo de correlações entre área e demais características

```sql

-- raio X área
SELECT ((N*SumTot)-(SumArea*SumPer))/(SQRT(N*SumArea2-SumArea*sumArea)*SQRT(N*SumPer2-SumPer*sumPer))
FROM   (SELECT SUM(Area) AS SumArea,
               SUM(Raio) AS sumPer,
               SUM(Area*Area) AS SumArea2,
               SUM(Raio*Raio) AS sumPer2,
               SUM(Area*Raio) AS sumTot,
               COUNT(*) AS N
         FROM   Paciente
        ) dados;

-- perímetro X área
SELECT ((N*SumTot)-(SumArea*SumPer))/(SQRT(N*SumArea2-SumArea*sumArea)*SQRT(N*SumPer2-SumPer*sumPer))
FROM   (SELECT SUM(Area) AS SumArea,
               SUM(Perimetro) AS sumPer,
               SUM(Area*Area) AS SumArea2,
               SUM(Perimetro*Perimetro) AS sumPer2,
               SUM(Area*Perimetro) AS sumTot,
               COUNT(*) AS N
         FROM   Paciente
        ) dados;

-- concavidade X área
SELECT ((N*SumTot)-(SumArea*SumPer))/(SQRT(N*SumArea2-SumArea*sumArea)*SQRT(N*SumPer2-SumPer*sumPer))
FROM   (SELECT SUM(Area) AS SumArea,
               SUM(Concavidade) AS sumPer,
               SUM(Area*Area) AS SumArea2,
               SUM(Concavidade*Concavidade) AS sumPer2,
               SUM(Area*Concavidade) AS sumTot,
               COUNT(*) AS N
         FROM   Paciente
        ) dados;

-- pontos concavos X área
SELECT ((N*SumTot)-(SumArea*SumPer))/(SQRT(N*SumArea2-SumArea*sumArea)*SQRT(N*SumPer2-SumPer*sumPer))
FROM   (SELECT SUM(Area) AS SumArea,
               SUM(PontosConcavos) AS sumPer,
               SUM(Area*Area) AS SumArea2,
               SUM(PontosConcavos*PontosConcavos) AS sumPer2,
               SUM(Area*PontosConcavos) AS sumTot,
               COUNT(*) AS N
         FROM   Paciente
        ) dados;

-- textura X área
SELECT ((N*SumTot)-(SumArea*SumPer))/(SQRT(N*SumArea2-SumArea*sumArea)*SQRT(N*SumPer2-SumPer*sumPer))
FROM   (SELECT SUM(Area) AS SumArea,
               SUM(Textura) AS sumPer,
               SUM(Area*Area) AS SumArea2,
               SUM(Textura*Textura) AS sumPer2,
               SUM(Area*Textura) AS sumTot,
               COUNT(*) AS N
         FROM   Paciente
        ) dados;

-- dimensão fractal X área
SELECT ((N*SumTot)-(SumArea*SumPer))/(SQRT(N*SumArea2-SumArea*sumArea)*SQRT(N*SumPer2-SumPer*sumPer))
FROM   (SELECT SUM(Area) AS SumArea,
               SUM(DimensaoFractal) AS sumPer,
               SUM(Area*Area) AS SumArea2,
               SUM(DimensaoFractal*DimensaoFractal) AS sumPer2,
               SUM(Area*DimensaoFractal) AS sumTot,
               COUNT(*) AS N
         FROM   Paciente
        ) dados;

-- suavidade X área
SELECT ((N*SumTot)-(SumArea*SumPer))/(SQRT(N*SumArea2-SumArea*sumArea)*SQRT(N*SumPer2-SumPer*sumPer))
FROM   (SELECT SUM(Area) AS SumArea,
               SUM(Suavidade) AS sumPer,
               SUM(Area*Area) AS SumArea2,
               SUM(Suavidade*Suavidade) AS sumPer2,
               SUM(Area*Suavidade) AS sumTot,
               COUNT(*) AS N
         FROM   Paciente
        ) dados;

-- simetria X área
SELECT ((N*SumTot)-(SumArea*SumPer))/(SQRT(N*SumArea2-SumArea*sumArea)*SQRT(N*SumPer2-SumPer*sumPer))
FROM   (SELECT SUM(Area) AS SumArea,
               SUM(Simetria) AS sumPer,
               SUM(Area*Area) AS SumArea2,
               SUM(Simetria*Simetria) AS sumPer2,
               SUM(Area*Simetria) AS sumTot,
               COUNT(*) AS N
         FROM   Paciente
        ) dados;
```


# XQuery

### UniProt - UniParc

#### Proposta 1:
É comum que algumas pessoas tenham alergia a alguns tipos de proteina, e isso é especialmente perigoso caso se essa proteina estiver relacionada a algum tipo de organismos contidos em vacinas, portanto uma pesquisa interessante é retornar todas as vacinas relacionadas a algum tipo de proteina

```xquery
let $doc := doc("uniparc-250.xml")
for $p in $doc//*:dbReference
   where contains(data($p/*:property[@type= 'gene_name']/@value), 'VAC') and $p/*:property[@type='NCBI_taxonomy_id']/@value != '10245'
return $p
```

#### Proposta 2:
Pode ser interessante analisarmos o total de proteinas relacionadas com a mesma taxonomia  

```xquery
let $doc := doc("uniparc-250.xml")
return count($doc//*:property[@type= 'NCBI_taxonomy_id'][@value= '32606'])
```

#### Proposta 3
Agrupar todas as proteinas com um mesmo proteoma também facilita uma série de analises
```xquery
let $doc := doc("uniparc-250.xml")
return $doc//*:property[@type= 'proteome_id'][@value= 'UP000304398']
```

#### Proposta 4
Encontrar todas as proteinas que tem mais de uma versão, pois isso significa que todas elas são sintéticas
```xquery
let $doc := doc("uniparc-250.xml")
return $doc//*:dbReference[@version_i > 1]
```

#### Proposta 5
Encontrar todas as proteinas que tem mais de uma versão, e estão ativas
```xquery
let $doc := doc("uniparc-250.xml")
return $doc//*:dbReference[@version_i > 1][@active='Y']
```

#### Proposta 6
Encontrar proteinas relacionadas a algum tipo de organismos contidos em vacinas
```xquery
let $doc := doc("uniparc-250.xml")
for $p in $doc//*:dbReference
   where contains(data($p/*:property[@type= 'gene_name']/@value), 'VAC')
return $p
```

### TB Database

#### Proposta 1:
Uma primeira informação que pode ser interessante na pesquisa de um cromossomo é sua complexidade. Uma possível forma de se medir isso utilizando a base em XML seria contar o número de propriedades que um determinado tipo de um cromossomo possui.

```xquery
let $doc := doc("file2.xml")

return count($doc//type[@value = 1]/property)
```

#### Proposta 2:
Contar o número de códons de parada em um determinado tipo do cromossomo.

```xquery
let $doc := doc("file2.xml")

for $c in ($doc//type[@value = 2])

return count($c/property[@feature = 'stop_codon'])
```

#### Proposta 3:
Encontrar propriedades a determinado gene em um dado tipo de cromossomo.

```xquery
let $doc := doc("file2.xml")

for $c in ($doc//type[@value = 4]/property)
where $c/@gene_id = 'RSP_2483'

return $c
```

#### Proposta 4:
Encontrar propriedades de um tipo de cromossomo que comecem após uma determinada posição do cromossomo.

```xquery
let $doc := doc("file2.xml")

for $c in ($doc//type[@value = 5]/property)
where $c/@start > 12000

return $c
```

#### Proposta 5:
Encontrar genes que remetem tanto a codons de início quanto a codons de parada em determinado tipo de cromossomo.

```xquery
let $doc := doc("file2.xml")

for $c in ($doc//type[@value = 6]/property),
    $p in ($doc//type[@value = 6]/property)
where $c/@feature = 'start_codon' and
      $p/@feature = 'stop_codon' and
      $c/@gene_id = $p/@gene_id

return data($c/@gene_id)
```

#### Proposta 6:
Encontrar propriedades de um tipo de cromossomo que ocupem um "espaço" pequeno, ou seja, ocupam um trecho pequeno do cromossomo.

```xquery
let $doc := doc("file2.xml")

for $c in ($doc//type[@value = 1]/property)
where $c/@end - $c/@start <= 10

return $c
```

# Cypher

### Virus-Host DB

#### Construção do grafo:
Com as seguintes instruções abaixo realizamos a construção dos grafos, o grafo é bem simples, basicamente retiramos os virus e os hospedeiros dos mesmos, adicionamos cada um deles apenas uma única vez no banco de dados. Um certo vírus possui uma aresta para seu hospedeiro (pode ter mais de um).

```cypher
USING PERIODIC COMMIT 10
LOAD CSV WITH HEADERS FROM
'file:///virushostdb.csv' AS line
WITH line
WHERE NOT line.host_tax_id IS NULL
MERGE (v:Virus {virus_name:line.virus_name,virus_id:line.virus_tax_id})
MERGE (h:Host{host_id:line.host_tax_id,host_name:line.host_name})
MERGE (v)-[i:Infects]->(h)
```

#### Proposta 1:
A primeira consulta que realizamos é determinar quantos vírus tem com seu hospedeiro o ser humano, ou seja a espécie "Homo sapiens".

```cypher
MATCH p=(v:Virus)-[r:Infects]->(h:Host {host_name:"Homo sapiens"}) 
RETURN count(v)
```

![](graph.png)

#### Proposta 2:
Aqui fazemos a análise de comunidades, apenas para percerbermos que nosso grafo não é bem adequado para esta análise,
uma vez que o número de comunidades é o mesmo que o número de nós, logo, cada nó é uma comunidade.

```cypher
CALL algo.unionFind('Host','Infects', {
  write: true,
  writeProperty: 'community'
})
YIELD nodes, setCount, loadMillis, computeMillis, writeMillis
```